In [15]:
import pandas as pd 
import seaborn as sns
from pandas import Series, DataFrame
from matplotlib import pyplot as plt
from sklearn import preprocessing
import pylab
import missingno as msno
import scipy.stats as stats
from scipy.stats import shapiro
import scipy
from numbers import Number
import statsmodels.stats.api as sms
import numpy as np 
from scipy.stats import levene
from sklearn.svm import SVC
from scipy.stats import f_oneway
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
import seaborn as sns
from skompiler import skompile
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from matplotlib import pyplot as plt
from sklearn import model_selection
from warnings import filterwarnings
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
filterwarnings("ignore")

In [2]:
data1 = pd.read_csv("hmelq.csv")
data = data1.copy()
data.dropna(inplace = True)
data.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
153,0,18200,94727.0,136877.0,DebtCon,Mgr,15.0,0.0,0.0,168.96,2.0,26.0,36.056
154,0,21700,79240.0,96784.0,DebtCon,Mgr,5.0,0.0,0.0,64.51,6.0,24.0,38.079
155,0,34100,241931.0,36486.0,DebtCon,Mgr,1.0,0.0,2.0,196.01,3.0,50.0,42.459
156,0,8400,62989.0,76718.0,HomeImp,Mgr,3.0,0.0,2.0,131.47,0.0,22.0,29.200
157,0,17400,25859.0,43684.0,DebtCon,Mgr,16.0,1.0,0.0,95.36,1.0,17.0,27.108


In [3]:
dummy = pd.get_dummies(data)
X = dummy.drop("bad", axis = 1)
y = dummy["bad"]
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.25, random_state = 42)


# SVC MODEL 

In [4]:
svc_model = SVC(kernel = "rbf").fit(X_train, y_train)

In [5]:
svc_model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [6]:
y_pred = svc_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9227722772277228

# Tuning

In [7]:
svc_params = {"C": [0.0001, 0.001, 0.1, 1, 5, 10 ,50 ,100],
             "gamma": [0.0001, 0.001, 0.1, 1, 5, 10 ,50 ,100]}

In [8]:
svc = SVC()
svc_cv_model = GridSearchCV(svc, svc_params, 
                         cv = 10, 
                         n_jobs = -1,
                         verbose = 2)

svc_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 64 candidates, totalling 640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 228 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 509 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed:   35.9s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.1, 1, 5, 10, 50, 100],
                         'gamma': [0.0001, 0.001, 0.1, 1, 5, 10, 50, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [9]:
print("En iyi parametreler: " + str(svc_cv_model.best_params_))

En iyi parametreler: {'C': 0.0001, 'gamma': 0.0001}


In [10]:
svc_tuned = SVC(C = 0.0001, gamma = 0.0001).fit(X_train, y_train)

In [11]:
y_pred = svc_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.9227722772277228


# KNN

In [12]:
data1 = pd.read_csv("hmelq.csv")
data = data1.copy()
data.dropna(inplace = True)
data.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
153,0,18200,94727.0,136877.0,DebtCon,Mgr,15.0,0.0,0.0,168.96,2.0,26.0,36.056
154,0,21700,79240.0,96784.0,DebtCon,Mgr,5.0,0.0,0.0,64.51,6.0,24.0,38.079
155,0,34100,241931.0,36486.0,DebtCon,Mgr,1.0,0.0,2.0,196.01,3.0,50.0,42.459
156,0,8400,62989.0,76718.0,HomeImp,Mgr,3.0,0.0,2.0,131.47,0.0,22.0,29.200
157,0,17400,25859.0,43684.0,DebtCon,Mgr,16.0,1.0,0.0,95.36,1.0,17.0,27.108


In [13]:
dummy = pd.get_dummies(data)
X = dummy.drop("bad", axis = 1)
y = dummy["bad"]
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.25, random_state = 42)

In [16]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
knn_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [17]:
y_pred = knn_model.predict(X_test)

In [18]:
accuracy_score(y_test, y_pred)

0.9227722772277228

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       466
           1       0.50      0.10      0.17        39

    accuracy                           0.92       505
   macro avg       0.71      0.55      0.56       505
weighted avg       0.90      0.92      0.90       505



# Tuning

In [20]:
knn_params = {"n_neighbors": np.arange(1,50), "metric"  :  ["minkowki", "euclidean", "manhattan" ]}

In [21]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv=10)
knn_cv.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'metric': ['minkowki', 'euclidean', 'manhattan'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [22]:
print("En iyi skor:" + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

En iyi skor:0.9107790170791217
En iyi parametreler: {'metric': 'manhattan', 'n_neighbors': 15}


In [23]:
knn = KNeighborsClassifier(metric='manhattan', n_neighbors=15)

In [24]:
knn_tuned = knn.fit(X_train, y_train)

In [25]:
knn_tuned.score(X_test, y_test)

0.9267326732673268

# Catboost

In [29]:
from catboost import CatBoostClassifier

In [30]:
cat_model = CatBoostClassifier().fit(X_train, y_train)

Learning rate set to 0.012295
0:	learn: 0.6790730	total: 64ms	remaining: 1m 3s
1:	learn: 0.6653024	total: 68ms	remaining: 33.9s
2:	learn: 0.6520232	total: 72.1ms	remaining: 24s
3:	learn: 0.6387608	total: 76.3ms	remaining: 19s
4:	learn: 0.6260466	total: 80.5ms	remaining: 16s
5:	learn: 0.6139957	total: 84.5ms	remaining: 14s
6:	learn: 0.6012506	total: 88ms	remaining: 12.5s
7:	learn: 0.5894019	total: 91.5ms	remaining: 11.3s
8:	learn: 0.5785514	total: 95.4ms	remaining: 10.5s
9:	learn: 0.5681672	total: 98.3ms	remaining: 9.73s
10:	learn: 0.5577601	total: 101ms	remaining: 9.08s
11:	learn: 0.5470630	total: 105ms	remaining: 8.63s
12:	learn: 0.5374239	total: 108ms	remaining: 8.22s
13:	learn: 0.5257013	total: 112ms	remaining: 7.9s
14:	learn: 0.5163719	total: 116ms	remaining: 7.62s
15:	learn: 0.5085850	total: 120ms	remaining: 7.35s
16:	learn: 0.4992611	total: 123ms	remaining: 7.11s
17:	learn: 0.4918311	total: 126ms	remaining: 6.89s
18:	learn: 0.4826634	total: 130ms	remaining: 6.72s
19:	learn: 0.474

In [31]:
y_pred = cat_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9663366336633663

# Tuning

In [32]:
catb_params = {
    'iterations': [200,500],
    'learning_rate': [0.01,0.05, 0.1],
    'depth': [3,5,8] }

In [33]:
catb = CatBoostClassifier()
catb_cv_model = GridSearchCV(catb, catb_params, cv=5, n_jobs = -1, verbose = 2)
catb_cv_model.fit(X_train, y_train)
catb_cv_model.best_params_

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  2.2min finished


0:	learn: 0.5812281	total: 13.1ms	remaining: 6.53s
1:	learn: 0.5065525	total: 17.8ms	remaining: 4.43s
2:	learn: 0.4490365	total: 25.3ms	remaining: 4.2s
3:	learn: 0.3917559	total: 33.2ms	remaining: 4.11s
4:	learn: 0.3491358	total: 43.4ms	remaining: 4.29s
5:	learn: 0.3119628	total: 52.5ms	remaining: 4.32s
6:	learn: 0.2863573	total: 61.2ms	remaining: 4.31s
7:	learn: 0.2649069	total: 68.7ms	remaining: 4.22s
8:	learn: 0.2473424	total: 79.9ms	remaining: 4.36s
9:	learn: 0.2341086	total: 88ms	remaining: 4.31s
10:	learn: 0.2161259	total: 95.4ms	remaining: 4.24s
11:	learn: 0.2092545	total: 105ms	remaining: 4.29s
12:	learn: 0.2004644	total: 114ms	remaining: 4.28s
13:	learn: 0.1951373	total: 122ms	remaining: 4.25s
14:	learn: 0.1887401	total: 129ms	remaining: 4.18s
15:	learn: 0.1849301	total: 136ms	remaining: 4.12s
16:	learn: 0.1809083	total: 144ms	remaining: 4.1s
17:	learn: 0.1758356	total: 154ms	remaining: 4.13s
18:	learn: 0.1708174	total: 162ms	remaining: 4.11s
19:	learn: 0.1673393	total: 171ms	

{'depth': 8, 'iterations': 500, 'learning_rate': 0.1}

In [34]:
catb_cv_model.best_params_

{'depth': 8, 'iterations': 500, 'learning_rate': 0.1}

In [35]:
catb = CatBoostClassifier(iterations = 200, 
                          learning_rate = 0.05, 
                          depth = 5)

catb_tuned = catb.fit(X_train, y_train)
y_pred = catb_tuned.predict(X_test)

0:	learn: 0.6384252	total: 5.4ms	remaining: 1.07s
1:	learn: 0.5923496	total: 8.4ms	remaining: 831ms
2:	learn: 0.5531549	total: 11.8ms	remaining: 772ms
3:	learn: 0.5170533	total: 14.6ms	remaining: 715ms
4:	learn: 0.4851083	total: 17.8ms	remaining: 693ms
5:	learn: 0.4524811	total: 20.7ms	remaining: 671ms
6:	learn: 0.4262403	total: 23.5ms	remaining: 648ms
7:	learn: 0.4028281	total: 26.2ms	remaining: 629ms
8:	learn: 0.3816458	total: 29.9ms	remaining: 634ms
9:	learn: 0.3613965	total: 33.5ms	remaining: 636ms
10:	learn: 0.3457999	total: 36.5ms	remaining: 627ms
11:	learn: 0.3296391	total: 39.3ms	remaining: 616ms
12:	learn: 0.3172065	total: 42.5ms	remaining: 611ms
13:	learn: 0.3066708	total: 45.3ms	remaining: 602ms
14:	learn: 0.2958377	total: 48.1ms	remaining: 594ms
15:	learn: 0.2876060	total: 51.1ms	remaining: 588ms
16:	learn: 0.2782833	total: 54.6ms	remaining: 587ms
17:	learn: 0.2716752	total: 57.9ms	remaining: 586ms
18:	learn: 0.2649424	total: 61.4ms	remaining: 585ms
19:	learn: 0.2584713	tot

In [36]:
y_pred = catb_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.9623762376237623